In [1]:
# Uncomment the below line for Google News
# !pip install -U gensim==3.8.3 

In [2]:
import gensim
from numpy import linalg
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm_notebook as tqdm
import time
from random import shuffle
import sys
import nltk 
from nltk.corpus import wordnet 
import gc
from collections import defaultdict
import random
import json
import os

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile

from gensim.test.utils import datapath

# This is for the plotly subplots
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import random
import scipy


!pip install ipython-autotime
%load_ext autotime

You should consider upgrading via the '/Users/stjepankusenic/POLAR_WEBE/venv/bin/python -m pip install --upgrade pip' command.
time: 1.67 ms (started: 2022-06-09 17:13:50 +02:00)


**Run this cell only for Glove Wiki**

In [3]:
def unzipping_file():
  !wget https://nlp.stanford.edu/data/glove.6B.zip
  !unzip glove*.zip # When it is unzipped you can see the different set of dimension it can have that is 25d, 50d, 100d or 200d

time: 763 µs (started: 2022-06-09 17:13:50 +02:00)


**Run this cell only for Google News**

In [4]:
# Uncomment below lines if running for google news
# In file_path variable line give the filepath of Google news bin file
#file_path = ""
#model_gn = gensim.models.KeyedVectors.load_word2vec_format(file_path,binary=True)
#current_model = model_gn

time: 334 µs (started: 2022-06-09 17:13:50 +02:00)


In [5]:
def model_initialization(unzippedname, new_name):
  model_glove = glove2word2vec(unzippedname,new_name)
  model_glove = gensim.models.KeyedVectors.load_word2vec_format(new_name, binary=False)
  return model_glove

time: 512 µs (started: 2022-06-09 17:13:50 +02:00)


In [6]:
def generate_norm_embedding(model, output_path):
    temp_file = open(output_path,'wb')
    temp_file.write(str.encode(str(len(model.vocab))+' '+str(model.vector_size)+'\n'))
    
    for each_word in tqdm(model.vocab):
        temp_file.write(str.encode(each_word+' '))  
        temp_file.write(model[each_word]/linalg.norm(model[each_word]))
        temp_file.write(str.encode('\n'))
    
    
    temp_file.close()
    return output_path

time: 1.09 ms (started: 2022-06-09 17:13:50 +02:00)


In [7]:
def load_normalized_embedding(output_file_name):
  return gensim.models.KeyedVectors.load_word2vec_format(output_file_name,binary=True)

time: 399 µs (started: 2022-06-09 17:13:50 +02:00)


**Loading the Antonym**

In [8]:
def load_antonyms(list_new,current_model):
  list_new= list(dict.fromkeys(list_new).keys())

  similarity_matrix = defaultdict(list)
  for each_pair in tqdm(list_new):
      word1 = each_pair[0]
      word2 = each_pair[1]
      if word1 < word2:
          similarity_matrix[word1].append(word2)
      else:
          similarity_matrix[word2].append(word1)

  all_similarity = defaultdict(dict)
  for each_key in tqdm(similarity_matrix):
      for each_value in similarity_matrix[each_key]:
  #         cosine_similarity([current_model[each_key]]
          try:
            all_similarity[each_key][each_value] = abs(cosine_similarity([current_model[each_key]],[current_model[each_value]])[0][0])
          except:
            print("The word is not in the vocabulary : ",each_key," ",each_value)

  final_list = []
  for index_counter, each_key in enumerate(tqdm(all_similarity)):
  #     print(each_key,all_similarity[each_key])
      listofTuples = sorted(all_similarity[each_key].items() ,  key=lambda x: x[1])
  #     print(listofTuples)
      final_list.append((each_key, listofTuples[0][0]))
  print(len(final_list))

  list_antonym = final_list
  
  return list_antonym

time: 2.47 ms (started: 2022-06-09 17:13:50 +02:00)


**Find the antonym difference vectors**

In [9]:
def antonym_difference_vector(list_antonym,current_model):
  num_antonym  = len(list_antonym)
  antonymy_vector = []
  for each_word_pair in list_antonym:
      antonymy_vector.append(current_model[each_word_pair[0]]- current_model[each_word_pair[1]])
      
  antonymy_vector = np.array(antonymy_vector)
  print(antonymy_vector.shape)
  print(antonymy_vector)
  return antonymy_vector

time: 1.04 ms (started: 2022-06-09 17:13:50 +02:00)


In [10]:
def generate_embedding_path(current_model, embedding_path, binary, antonym_vector, curr_dim,antonymy_vector):
    curr_antonym_vector = antonymy_vector[antonym_vector[:curr_dim]]
    curr_antonymy_vector_inverse = np.linalg.pinv(np.transpose(curr_antonym_vector))
    new_embedding_dict = transform_to_antonym_space(current_model, embedding_path, binary,curr_antonymy_vector_inverse)

    return new_embedding_dict

time: 736 µs (started: 2022-06-09 17:13:50 +02:00)


In [11]:
def transform_to_antonym_space(current_model, output_file_path, binary, current_antonymy_vector_inverse):

    temp_dict = dict()

    embedding_size = current_antonymy_vector_inverse.shape[0]   ##CHANGE THIS ACCORDINGLY!!!
    #print('New model size is',len(current_model), embedding_size)

    temp_file = None
    
    if binary:
        temp_file = open(output_file_path,'wb')
        temp_file.write(str.encode(str(len(current_model))+' '+str(embedding_size)+'\n'))
    else:
        temp_file = open(output_file_path,'w')
        temp_file.write(str(len(current_model))+' '+str(embedding_size)+'\n')

    total_words = 0
    for each_word in tqdm(current_model):
        total_words += 1
        if binary:
            temp_file.write(str.encode(each_word+' '))
        else:
            temp_file.write(each_word+' ')

        new_vector = np.matmul(current_antonymy_vector_inverse,current_model[each_word])

        new_vector = new_vector/linalg.norm(new_vector)
        temp_dict[each_word] = new_vector
        
        if binary:
            temp_file.write(new_vector)
            temp_file.write(str.encode('\n'))
        else:
            temp_file.write(str(new_vector))
            temp_file.write('\n')


    temp_file.close()
    return temp_dict

time: 2.39 ms (started: 2022-06-09 17:13:50 +02:00)


In [12]:
def make_polar_dict(company_name, antonym, embedding, top_n = False, n = 10):
  temp_dict = dict()
  temp_polar = embedding[company_name]

  if top_n:
    idx = np.argsort([abs(x) for x in temp_polar])[-n:]
    for i in idx:
      print(antonym[i],temp_polar[i],'\n')


  if len(antonym) == len(temp_polar):
    for a in range(len(antonym)):
      temp_dict[antonym[a]] = temp_polar[a]
    return temp_dict

time: 1.84 ms (started: 2022-06-09 17:13:50 +02:00)


In [13]:
def download_csv_file(df,filename):
  df.to_csv(filename)
  files.download(filename)

time: 494 µs (started: 2022-06-09 17:13:50 +02:00)


**The below function will input the antonym pair from the user**

In [14]:
def user_antonym_input(list_new,user_list):
  list_new.append(user_list)
  return list_new

time: 607 µs (started: 2022-06-09 17:13:50 +02:00)


In [15]:
def clear_tuple(tuples):
  temp = list(tuples)
  temp.clear()
  tuples = tuple(temp)
  return tuples

time: 717 µs (started: 2022-06-09 17:13:50 +02:00)


**Below cell is to create the random Word list**

In [16]:
def random_word_list(number_of_elements):  
  list_antonym = []  
  with open('../data/raw/Antonym_sets/LenciBenotto.val') as fp:  
      for line in fp:
          parts = line.split() # The line is split into 4 parts as there are 3 space in between
          if parts[3]=='antonym': # Here we if the the 4th part that is parts[3] in the line is antonym then that line is selected
              word1 = parts[0].split('-')[0]
              word2 = parts[1].split('-')[0]
              if word1 in current_model and word2 in current_model: # If both the words are there in the model then it is appended to the list
                  list_antonym.append((word1.strip().lower(), word2.strip().lower()))
  
  return random.sample(list_antonym, number_of_elements)

time: 1.62 ms (started: 2022-06-09 17:13:50 +02:00)


In [17]:
def similar_word_to_give_antonym_pair(input_list,current_model):
  new_similar_list= [] 

  for i in input_list:
    antonym_pair = i
    first_word = antonym_pair[0]
    second_word = antonym_pair[1]

    similar_word_list1 = current_model.similar_by_word(first_word)
    similar_word_list2 = current_model.similar_by_word(second_word)  
    
    new_antonym_word1 = similar_word_list1[0][0]
    new_antonym_word2 = similar_word_list2[0][0]   
   
    antonym_tuple = (new_antonym_word1,new_antonym_word2)
    
    new_similar_list.append(antonym_tuple)
    clear_tuple(antonym_tuple)
    
  return new_similar_list

time: 864 µs (started: 2022-06-09 17:13:50 +02:00)


**Below cell will load the Glove Wiki model and unzip the model**

In [18]:
#unzipping_file()
#model_glove = model_initialization('glove.6B.300d.txt','gensim_glove.6B.300d.txt')
#path = generate_norm_embedding(model_glove,'glove_norm_wiki.mod')
#current_model  = load_normalized_embedding(path)
model_gn = gensim.models.KeyedVectors.load_word2vec_format('../data/raw/glove_norm_300.mod',binary=True)
current_model = model_gn

time: 13.3 s (started: 2022-06-09 17:13:50 +02:00)


**Below cell will load the model trained on Google News and unzip the model**

In [19]:
# Uncomment below lines if running for google news

#model_glove = model_initialization('glove.6B.300d.txt','gensim_glove.6B.300d.txt')
#path = generate_norm_embedding(model_glove,'glove_norm_wiki.mod')
#current_model  = load_normalized_embedding(path)
#model_gn = gensim.models.KeyedVectors.load_word2vec_format('../data/raw/GoogleNews-vectors-negative300.mod',binary=True)
#current_model = model_gn

time: 319 µs (started: 2022-06-09 17:14:03 +02:00)


**Random List**

In [20]:
list_powerdistance_random =[('make', 'break'), ('cameraman', 'playwright'), ('mystical', 'factual'), ('promotional', 'defamation'), ('iconic', 'unknown')]
list_individualism_random = [('lop', 'secure'), ('shah', 'poor'), ('pneumatic', 'solid'), ('interpret', 'misinterpret'), ('confer', 'refuse')]
list_masculinity_random = [('try', 'abstain'), ('fatalistic', 'freewill'), ('knowledgeable', 'uninformed'), ('confine', 'free'), ('fan', 'warm')]
list_longterm_random = [('innovator', 'follower'), ('sensory', 'numb'), ('hedge', 'squander'), ('arachnid', 'serpent'), ('disclose', 'secrete')]
list_indulgence_random = [('diagnose', 'sicken'), ('intercourse', 'disconnection'), ('sensory', 'sensorial'), ('emasculate', 'strengthen'), ('metropolitan', 'rural')]
list_uncertainity_avoidance_random = [('stretcher', 'compressor'), ('amalgamate', 'separate'), ('caretaker', 'assailant'), ('taker', 'violator'), ('contaminate', 'sterilize')]

time: 1.11 ms (started: 2022-06-09 17:14:03 +02:00)


**Human generated list**

In [21]:
list_powerdistance =[('hierarchical','non-hierarchical'),('superior','equal'),('leader','subordinate'),('inequality','equality'),('autocrat','democrat')]
list_individualism = [('individuality','community'),('self-interest','harmony'),('tasks','relationships'),('individual','groups'),('particularism','universalism')]
list_masculinity = [('achievement', 'support'),('competitive', 'caring'),('assertive', 'submissive'),('ambitious', 'unambitious'),('sucess','cooperation')]
list_longterm = [('pragmatic','normative'),('progress','preserve'),('adapt','conserve'),('developing','stable'),('advance','retain')]
list_indulgence = [('fulfillment','restriction'),('satisfaction','limitation'),('liberty','moderation'),('expand','direct'),('freedom','regulation')]
list_uncertainity_avoidance = [('clarity','complexity'),('clear','ambiguous'),('certain','uncertain'),('uniformity','diversity'),('agreement','variation')]


time: 1.11 ms (started: 2022-06-09 17:14:03 +02:00)


**The nearest pairs to Random list suggested by the pretrained model**

In [22]:
similar_list_powerdistance_random = similar_word_to_give_antonym_pair(list_powerdistance_random,current_model)
similar_list_individualism_random = similar_word_to_give_antonym_pair(list_individualism_random,current_model)
similar_list_masculinity_random = similar_word_to_give_antonym_pair(list_masculinity_random,current_model)
similar_list_longterm_random = similar_word_to_give_antonym_pair(list_longterm_random,current_model)
similar_list_indulgence_random = similar_word_to_give_antonym_pair(list_indulgence_random,current_model)
similar_list_uncertainity_avoidance_random = similar_word_to_give_antonym_pair(list_uncertainity_avoidance_random,current_model)

time: 27.2 s (started: 2022-06-09 17:14:03 +02:00)


**The nearest pairs to human made list suggested by the pretrained model**

In [23]:
similar_list_powerdistance = similar_word_to_give_antonym_pair(list_powerdistance,current_model)
similar_list_individualism = similar_word_to_give_antonym_pair(list_individualism,current_model)
similar_list_masculinity = similar_word_to_give_antonym_pair(list_masculinity,current_model)
similar_list_longterm = similar_word_to_give_antonym_pair(list_longterm,current_model)
similar_list_indulgence = similar_word_to_give_antonym_pair(list_indulgence,current_model)
similar_list_uncertainity_avoidance = similar_word_to_give_antonym_pair(list_uncertainity_avoidance,current_model)

time: 7.94 s (started: 2022-06-09 17:14:31 +02:00)


**In the below cell give the list to create the polar embedding**


In [24]:
# If we give list_masculinity then the polar embedding of the company and the 5 antonym pair in the list_masculinity will be considered
list_new = list_masculinity

time: 539 µs (started: 2022-06-09 17:14:38 +02:00)


In [25]:
list_antonym  = load_antonyms(list_new,current_model)
len(list_new)
antonymy_vector = antonym_difference_vector(list_antonym,current_model)

/var/folders/xy/yfm2dhtj3jvfy85hqy16wmk00000gn/T/ipykernel_23237/1157002585.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for each_pair in tqdm(list_new):


  0%|          | 0/5 [00:00<?, ?it/s]

/var/folders/xy/yfm2dhtj3jvfy85hqy16wmk00000gn/T/ipykernel_23237/1157002585.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for each_key in tqdm(similarity_matrix):


  0%|          | 0/5 [00:00<?, ?it/s]

/var/folders/xy/yfm2dhtj3jvfy85hqy16wmk00000gn/T/ipykernel_23237/1157002585.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index_counter, each_key in enumerate(tqdm(all_similarity)):


  0%|          | 0/5 [00:00<?, ?it/s]

5
(5, 300)
[[-0.07274781  0.02826376 -0.05573687 ... -0.0699044  -0.0272214
   0.05764209]
 [-0.03125679  0.0518945  -0.04949443 ...  0.08807085  0.05805586
  -0.04926867]
 [ 0.04541304  0.10594551  0.07777175 ...  0.10511187  0.04088513
   0.06519555]
 [ 0.00272555  0.1086304   0.007989   ...  0.06512681  0.09706157
   0.05637065]
 [ 0.03257257 -0.10196585  0.0694065  ...  0.08014861 -0.00261596
  -0.00712908]]
time: 247 ms (started: 2022-06-09 17:14:39 +02:00)


Reading the Fortune 500 companies.csv file

In [26]:
company = pd.read_csv('../data/raw/International_Fortune_GloVe.csv',encoding= 'unicode_escape')
name_list = company['0']

time: 23.7 ms (started: 2022-06-09 17:14:39 +02:00)


**Below Cell will see the companies that is there in the vocabulary of the pretrained model**

In [27]:
name_word_embedding = dict()
new_name_list=[]
for name in name_list:
  try:
    name_word_embedding[name] = current_model[name]
    new_name_list.append(name)
  except:
    print("The company is not there: ",name)

print(new_name_list)

The company is not there:  nestlÃ©
The company is not there:  telefÃ³nica
The company is not there:  l'orÃ©al
['walmart', 'amazon', 'apple', 'volkswagen', 'mckesson', 'amerisourcebergen', 'bp', 'alphabet', 'exxonmobil', 'daimler', 'at&t', 'cigna', 'microsoft', 'glencore', 'allianz', 'kroger', 'homedepot', 'jpmorganchase', 'axa', 'generalmotors', 'anthem', 'mitsubishi', 'centene', 'fanniemae', 'comcast', 'itochu', 'chevron', 'bankofamerica', 'target', 'citigroup', 'gazprom', 'facebook', 'sony', 'hitachi', 'carrefour', 'tesco', 'aeon', 'wellsfargo', 'generalelectric', 'intel', 'sinopharm', 'humana', 'cofco', 'mitsui', 'enel', 'lukoil', 'sk', 'e.on', 'pepsico', 'albertsons', 'basf', 'fedex', 'metlife', 'freddiemac', 'waltdisney', 'siemens', 'sinochem', 'engie', 'panasonic', 'aviva', 'marubeni', 'aegon', 'boeing', 'unilever', 'airbus', 'petrobras', 'hp', 'prudential', 'fortum', 'arcelormittal', 'sysco', 'jbs', 'morganstanley', 'ptt', 'indianoil', 'vinci', 'kia', 'eni', 'kddi', 'novartis', 

In [28]:
random_antonym_vector = [i for i in range(len(antonymy_vector))]
random.shuffle(random_antonym_vector)
print(len(random_antonym_vector))
name_new_embedding = generate_embedding_path(name_word_embedding, 'test_run',True ,random_antonym_vector,500,antonymy_vector)

5


/var/folders/xy/yfm2dhtj3jvfy85hqy16wmk00000gn/T/ipykernel_23237/3001024653.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for each_word in tqdm(current_model):


  0%|          | 0/164 [00:00<?, ?it/s]

time: 114 ms (started: 2022-06-09 17:14:39 +02:00)


In [29]:
antonym_500 = [list_antonym[x] for x in random_antonym_vector[:500]]

time: 418 µs (started: 2022-06-09 17:14:39 +02:00)


**Polar Embedding -> DataFrame**

In [30]:
df = dict()
for t in new_name_list:
  df[t] = make_polar_dict(t, antonym_500, name_new_embedding)

new_df = pd.DataFrame(df).transpose()


# change columns to better read names
new_columns = []

for pair in antonym_500:
  temp = pair[0]+''+pair[1]
  new_columns.append(temp)

new_df.columns = new_columns

time: 14.6 ms (started: 2022-06-09 17:14:39 +02:00)


**In the first run uncomment the cell below as the copy of new_df is made and it is into the concat_polar_embedding_df we concatinate the polar embedding**

In [31]:
concat_polar_embedding_df = new_df.copy()

time: 687 µs (started: 2022-06-09 17:14:39 +02:00)


**In the second run onwards comment the previous cell and uncomment the lower cell**

In [32]:
#concat_polar_embedding_df = concat_polar_embedding_df.join(new_df)

time: 263 µs (started: 2022-06-09 17:14:39 +02:00)


**Below cell is to download the file once it is ready**

In [33]:
'''
concat_polar_embedding_df.to_csv('concat_polar_embedding_df.csv',header=True)
'''

"\nconcat_polar_embedding_df.to_csv('concat_polar_embedding_df.csv',header=True)\n"

time: 7.39 ms (started: 2022-06-09 17:14:39 +02:00)
